In [11]:
import sys
import math
import numpy as np
import torch
sys.path.append('../')

from src.graph_utils import partition_graph_by_lonlat
import networkx as nx
from jurbey.jurbey import JURBEY

with open("../data/berlin.jurbey", 'rb') as tempf:
    g = JURBEY.load(tempf.read())
print(g.number_of_nodes())
g_partition = partition_graph_by_lonlat(g)

A = nx.adjacency_matrix(g_partition)


290100


## Load in timeseries data

In [3]:
import math
id_to_idx = {}
df = pandas.read_csv("../data/timeseries_speed_april_first_week.csv")

df.head()


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,136,137,138,139,140,141,142,143,from_node,to_node
0,result.average,NaN,NaN,NaN,10.854457,NaN,5.680978,7.036838,1.499997,10.088634,...,10.581123,NaN,NaN,NaN,NaN,NaN,10.850843,10.036408,628154368,1023689595
1,result.average,NaN,NaN,NaN,10.854457,NaN,5.680978,7.036838,1.499997,10.088634,...,10.581123,NaN,NaN,NaN,NaN,NaN,10.850843,10.036408,628154368,1023689595
2,result.average,NaN,NaN,NaN,NaN,NaN,10.486210,NaN,NaN,10.937218,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,527147009,27537239
3,result.average,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,527147009,26908815
4,result.average,NaN,NaN,NaN,10.468811,10.636621,10.288534,10.617513,1.758539,3.404401,...,6.098767,10.257657,10.586166,NaN,10.669476,4.313417,10.392901,10.518310,628154370,3804638178


In [4]:
print(f"Shape of data: {df.shape}. Missing in data: {df.isnull().sum().sum()}.")

Shape of data: (6162, 147). Missing in data: 818224.


## remove road segments with > 50% missing data

In [54]:
df = pd.read_csv("../data/timeseries_speed_april_first_week.csv")


df = df.T
print(f"original number of road segment: {df.shape[1]}")
df = df.loc[:, df.isnull().mean() < 0.5]
print(f"original number of road segment after filtering: {df.shape[1]}")
df = df.T
df = df.loc[:, df.columns != 'Unnamed: 0']
df = df.loc[:, df.columns != 'from_node']
df = df.loc[:, df.columns != 'to_node']
print(df[0:10])
df = df.T

original number of road segment: 6162
original number of road segment after filtering: 213
           0        1    2        3        4        5          6         7  \
4        NaN      NaN  NaN  10.4688  10.6366  10.2885    10.6175   1.75854   
7    10.2788      NaN  NaN      NaN  10.8314      NaN    3.14074   2.07388   
16   10.8814      NaN  NaN      NaN  10.9489  10.1905    8.48206   10.6813   
124  10.2644      NaN  NaN  10.5666      NaN  7.37511   0.915255   10.1382   
157      NaN      NaN  NaN      NaN  10.8314      NaN    7.53377   5.43425   
204      NaN  10.0555  NaN      NaN      NaN      NaN  0.0954053    10.709   
351      NaN      NaN  NaN  10.4688  10.4049  7.20193    7.68655   7.31936   
352      NaN      NaN  NaN   10.471  10.3797  0.40066   0.149782  0.486138   
379  10.3584      NaN  NaN      NaN  10.6964  6.91769    5.38908   10.8364   
389      NaN      NaN  NaN  10.4688    10.38   7.0872    8.65096   10.2432   

            8        9  ...      134        135   

## Interpolate misisng values with Krogh

In [55]:
for column in df:
    df[column] = pd.to_numeric(df[column])

df = df.interpolate(method='nearest', axis=1)

df = df.fillna(method='backfill')
df[0:10]

,4,7,16,124,157,204,351,352,379,389,...,5953,5956,5964,5970,5981,5988,6002,6004,6121,6157
0,10.468811,10.278846,10.881422,10.264438,10.264438,10.264438,10.358411,10.358411,10.358411,10.358411,...,10.670639,10.670639,10.670639,10.670639,10.670639,10.523302,10.523302,10.523302,10.523302,10.446166
1,10.468811,10.468811,10.468811,10.566648,10.566648,10.055472,10.359838,10.359838,10.359838,10.359838,...,10.249640,10.249640,10.249640,10.249640,10.523302,10.523302,10.523302,10.523302,10.523302,10.446166
2,10.468811,10.468811,10.468811,10.566648,10.566648,10.566648,10.468811,10.471016,10.468811,10.468811,...,3.183892,3.183892,3.183892,3.183892,3.183892,3.183892,3.183892,3.183892,3.183892,3.183892
3,10.468811,10.468811,10.468811,10.566648,10.566648,10.566648,10.468811,10.471016,10.468811,10.468811,...,5.404969,5.404969,6.499586,6.499586,10.489977,10.489977,5.641341,1.774943,6.938167,6.938167
4,10.636621,10.831409,10.948903,10.831409,10.831409,10.831409,10.404854,10.379701,10.696351,10.380026,...,4.587224,1.906247,0.421084,0.421084,10.896709,10.346092,4.223832,0.390303,2.033530,2.033530
5,10.288534,10.288534,10.190479,7.375108,7.375108,7.375108,7.201934,0.400660,6.917693,7.087202,...,1.704999,1.285552,10.338850,0.655349,7.138202,0.115042,1.362726,0.149050,10.398745,1.690993
6,10.617513,3.140737,8.482057,0.915255,7.533769,0.095405,7.686549,0.149782,5.389075,8.650964,...,0.000000,0.658714,4.124535,0.509908,7.888281,6.685049,0.765714,1.420218,10.531851,0.373156
7,1.758539,2.073876,10.681286,10.138236,5.434254,10.709016,7.319358,0.486138,10.836391,10.243176,...,1.547999,1.275324,10.092655,0.989406,10.486895,9.096087,1.170159,0.500815,2.105265,0.413589
8,3.404401,0.385263,5.028228,1.443512,10.757339,2.713814,5.894384,1.700133,7.809842,8.313960,...,0.469963,0.469963,10.605759,1.232956,8.237365,2.376006,4.479992,1.920087,1.315475,1.315475
9,10.224568,4.104480,10.550042,1.394967,10.350923,10.953392,10.279088,10.034991,10.573360,10.200276,...,0.235419,2.096244,1.971357,1.454085,10.247256,10.665007,1.806557,0.668753,10.559178,0.814330


## Prepare training datasets

In [56]:

class DataLoader():
    """A class for loading and transforming data for the lstm model"""

    def __init__(self, dataframe, split):
        print(f"length of dataframe: {len(dataframe)}")
        i_split = int(len(dataframe) * split)
        self.data_train = dataframe.values[:i_split]
        print(f"size of training data: {len(self.data_train)}")
        self.data_test  = dataframe.values[i_split:]
        self.len_train  = len(self.data_train)
        self.len_test   = len(self.data_test)
        self.len_train_windows = None

    def get_test_data(self, seq_len, normalise):
        '''
        Create x, y test data windows
        Warning: batch method, not generative, make sure you have enough memory to
        load data, otherwise reduce size of the training split.
        '''
        data_windows = []
        for i in range(self.len_test - seq_len):
            data_windows.append(self.data_test[i:i+seq_len])

        data_windows = np.array(data_windows).astype(float)
        data_windows = self.normalise_windows(data_windows, single_window=False) if normalise else data_windows
        x = data_windows[:,:-1]
        y = data_windows[:,-1:]
        
        x = torch.from_numpy(np.swapaxes(x,1,2))
        y = torch.from_numpy(np.swapaxes(y,1,2))
        return x,y

    def get_train_data(self, seq_len, normalise):
        '''
        Create x, y train data windows
        Warning: batch method, not generative, make sure you have enough memory to
        load data, otherwise use generate_training_window() method.
        '''
        data_x = []
        data_y = []
        for i in range(self.len_train - seq_len):
            x, y = self._next_window(i, seq_len, normalise)
            data_x.append(x)
            data_y.append(y)
        data_x = np.array(data_x, dtype=np.float32)
        data_y = np.array(data_y, dtype=np.float32)
        return torch.from_numpy(np.swapaxes(data_x,1,2)), torch.from_numpy(np.swapaxes(data_y,1,2))
        

    def generate_train_batch(self, seq_len, batch_size, normalise):
        '''Yield a generator of training data from filename on given list of cols split for train/test'''
        i = 0
        while i < (self.len_train - seq_len):
            x_batch = []
            y_batch = []
            for b in range(batch_size):
                if i >= (self.len_train - seq_len):
                    # stop-condition for a smaller final batch if data doesn't divide evenly
                    yield np.array(x_batch), np.array(y_batch)
                    i = 0
                x, y = self._next_window(i, seq_len, normalise)
                x_batch.append(x)
                y_batch.append(y)
                i += 1
            yield np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.float32)

    def _next_window(self, i, seq_len, normalise):
        '''Generates the next data window from the given index location i'''
        window = self.data_train[i:i+seq_len]
        window = self.normalise_windows(window, single_window=True)[0] if normalise else window
        x = window[:-1,:]
        y = window[-1:,:]
        return x, y

    def normalise_windows(self, window_data, single_window=False):
        '''Normalise window with a base value of zero'''
        normalised_data = []
        window_data = [window_data] if single_window else window_data
        for window in window_data:
            normalised_window = []
            for col_i in range(window.shape[1]):
                normalised_col = [((float(p) / float(window[0, col_i])) - 1) for p in window[:, col_i]]
                normalised_window.append(normalised_col)
            normalised_window = np.array(normalised_window).T # reshape and transpose array back into original multidimensional format
            normalised_data.append(normalised_window)
        return np.array(normalised_data)


In [57]:
print(len(df))
data = DataLoader(df, 0.8)

seq_len=10

144
length of dataframe: 144
size of training data: 115


In [58]:
x, y = data.get_train_data(seq_len=10, normalise=False)
x_valid, y_valid = data.get_test_data(seq_len=seq_len, normalise=False)

print(f"training data shapes: {x.shape}, {y.shape}")
print(f"validation data shapes: {x_valid.shape}, {y_valid.shape}")

training data shapes: torch.Size([105, 213, 9]), torch.Size([105, 213, 1])
validation data shapes: torch.Size([19, 213, 9]), torch.Size([19, 213, 1])


## Now start training

In [59]:
from test_tube import Experiment 
from pytorch_lightning import Trainer
import os

# PyTorch summarywriter with a few bells and whistles    
exp = Experiment(save_dir=os.getcwd())

# pass in experiment for automatic tensorboard logging.    
trainer = Trainer(experiment=exp, max_nb_epochs=1000, train_percent_check=1)


gpu available: False, used: False


In [60]:
look_back=9
train=torch.zeros(len(x),213,look_back+1)
train[:,:,:look_back]=x
train[:,:,look_back:(look_back+1)]=y

valid=torch.zeros(len(x_valid),213,look_back+1)
valid[:,:,:look_back]=x_valid
valid[:,:,look_back:(look_back+1)]=y_valid

In [61]:
from torch.utils.data import DataLoader
dataset = DataLoader(train, batch_size=10, shuffle=True)
print(len(dataset))

11


In [65]:
import imp  
import src.tgcn.layers.lstm as l
l = imp.reload(l)
LSTMs = l.LSTMs
model = LSTMs(input_dim=9, hidden_dim=32, data=train, validation_data=valid)
trainer.fit(model)


NameError: name 'indices' is not defined